In [803]:
# %%capture
# %pip install copulas==0.8.0

In [804]:
# !gdown 1LK03a5IbPiV6RosdTR15gjZpLIg3YnBl

# Required Libs

In [805]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

import pandas as pd
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE # TSNE module
from sklearn.decomposition import PCA

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, balanced_accuracy_score

In [806]:
df = pd.read_excel('/content/TrainDataset2023.xls')
df.drop(['ID'], axis=1, inplace = True) # Drop ID # 'RelapseFreeSurvival (outcome)'

# Drop NaN vals
for col in df.columns:
  df.drop(df[df[col] == 999].index, inplace=True)

df = df[df.columns].reset_index(drop=True)
df = df.drop(['pCR (outcome)'], axis=1)

df_ = df.drop(['RelapseFreeSurvival (outcome)'], axis=1)

In [807]:
df.head()
df_train, df_test = train_test_split(df, test_size=0.2, random_state=123) # Classlification Label
df_train_no_leb, df_test_no_leb = df_train[df_train.columns[1:]], df_test[df_train.columns[1:]]

In [808]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

columns_minmax_encode = ['Age'] + df_train_no_leb.columns[10:].tolist()
columns_onehot_encode = df_train_no_leb.columns[1:5].tolist() + df_train_no_leb.columns[7:9].tolist()
columns_label_encode  = df_train_no_leb.columns[5:7].tolist() + [df_train_no_leb.columns[9]]

# Min-Max Scaling
scaler = MinMaxScaler()
df_train_no_leb[columns_minmax_encode] = scaler.fit_transform(df_train_no_leb[columns_minmax_encode])

# One-Hot Encoding
df_train_no_leb = pd.get_dummies(df_train_no_leb, columns=columns_onehot_encode)

# Label Encoding
label_encoder = LabelEncoder()
for col in columns_label_encode:
    df_train_no_leb[col] = label_encoder.fit_transform(df_train_no_leb[col])

<ipython-input-808-2412451becf4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_no_leb[columns_minmax_encode] = scaler.fit_transform(df_train_no_leb[columns_minmax_encode])


# Feature Selection

In [809]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

columns_minmax_encode = ['Age'] + df_train_no_leb.columns[10:].tolist()
columns_onehot_encode = df_train_no_leb.columns[1:5].tolist() + df_train_no_leb.columns[7:9].tolist()
columns_label_encode  = df_train_no_leb.columns[5:7].tolist() + [df_train_no_leb.columns[9]]

# Min-Max Scaling
scaler = MinMaxScaler()
df_train_no_leb[columns_minmax_encode] = scaler.fit_transform(df_train_no_leb[columns_minmax_encode])

# One-Hot Encoding
df_train_no_leb = pd.get_dummies(df_train_no_leb, columns=columns_onehot_encode)

# Label Encoding
label_encoder = LabelEncoder()
for col in columns_label_encode:
    df_train_no_leb[col] = label_encoder.fit_transform(df_train_no_leb[col])

In [810]:
# Selected Features by Correlation
corr_matrix = df_train.corr()

# list of selected features
threshold = 0.05
selected = corr_matrix[corr_matrix['RelapseFreeSurvival (outcome)'] > threshold]['RelapseFreeSurvival (outcome)']
selected_feas = corr_matrix[corr_matrix['RelapseFreeSurvival (outcome)'] > threshold].axes[0].tolist()[1:] # select just features names
selected_feas

['HER2',
 'original_shape_Sphericity',
 'original_firstorder_10Percentile',
 'original_firstorder_Kurtosis',
 'original_firstorder_Minimum',
 'original_firstorder_Uniformity',
 'original_glcm_ClusterShade',
 'original_glcm_Id',
 'original_glcm_Idm',
 'original_glcm_Idmn',
 'original_glcm_Idn',
 'original_glcm_Imc1',
 'original_glcm_JointEnergy',
 'original_glcm_MaximumProbability',
 'original_gldm_LargeDependenceEmphasis',
 'original_gldm_LargeDependenceLowGrayLevelEmphasis',
 'original_gldm_SmallDependenceEmphasis',
 'original_gldm_SmallDependenceLowGrayLevelEmphasis',
 'original_glszm_GrayLevelNonUniformityNormalized',
 'original_glszm_LowGrayLevelZoneEmphasis',
 'original_glszm_SmallAreaEmphasis',
 'original_glszm_SmallAreaHighGrayLevelEmphasis',
 'original_glszm_SmallAreaLowGrayLevelEmphasis',
 'original_glszm_ZonePercentage',
 'original_ngtdm_Coarseness']

In [832]:
df_test[['RelapseFreeSurvival (outcome)'] + selected_feas].reset_index(drop=True).to_csv('/content/original_test.csv', index = False)

In [811]:
# Set train test validation
dftrain = df_train[selected_feas].reset_index(drop=True).values.tolist()[40:]
dfval   = df_train[selected_feas].reset_index(drop=True).values.tolist()[0:40]
dftest  = df_test[selected_feas].reset_index(drop=True).values.tolist()

y_train = df_train['RelapseFreeSurvival (outcome)'].reset_index(drop=True)[40:]
y_val   = df_train['RelapseFreeSurvival (outcome)'].reset_index(drop=True)[0:40]
y_test  = df_test['RelapseFreeSurvival (outcome)'].reset_index(drop=True)

In [812]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MLP_Regressor(nn.Module):
    def __init__(self, input_shape):
        super(MLP_Regressor, self).__init__()

        self.cf1 = nn.Linear(input_shape, 32)
        self.cf2 = nn.Linear(32, 16)
        self.cf3 = nn.Linear(16, 1)

        self.relu    = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.batchnorm2 = nn.BatchNorm1d(8)

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)

    def forward(self, x):
        x  = self.relu(self.batchnorm1(self.cf1(x)))
        x  = self.dropout1(x)
        x  = self.relu(self.cf2(x))
        # x  = self.dropout2(x)
        x  = self.cf3(x)

        return x


X_train = torch.tensor(dftrain, dtype=torch.float32)
X_val   = torch.tensor(dfval, dtype=torch.float32)
X_test  = torch.tensor(dftest, dtype=torch.float32)

y_train = torch.tensor(y_train.tolist(), dtype=torch.float32)
y_val   = torch.tensor(y_val.tolist(), dtype=torch.float32)
y_test  = torch.tensor(y_test.tolist(), dtype=torch.float32)

In [813]:
def MLP_RegressorLoop(model, criterion, optimizer, scheduler, epochs, X_train, y_train, X_val, y_val):
    val_loss_his = []
    train_loss_his = []
    count = 1
    MODEL = []
    for eph in range(epochs):
        loss_epoch_train = []
        loss_epoch_val = []

        model.train()

        optimizer.zero_grad()

        X_train, y_train = X_train.to(device), y_train.to(device)

        output = model(X_train)

        loss = torch.sqrt(criterion(output, y_train.unsqueeze(1)))

        loss_epoch_train.append(loss.cpu().detach().numpy())

        loss.backward()
        optimizer.step()

        train_loss_his.append(np.mean(loss_epoch_train))

        # Run the validation batches
        model.eval()
        with torch.no_grad():

            X_val, y_val = X_val.to(device), y_val.to(device)

            out_val = model(X_val)

            loss = torch.sqrt(criterion(out_val.cpu(), y_val.cpu()))

            loss_epoch_val.append(loss.item())

        val_loss_his.append(np.mean(loss_epoch_val))
        scheduler.step(np.mean(loss_epoch_val))

        print('RMSE', loss.numpy())

        if eph > 1:
            if np.mean(loss_epoch_val) < min(val_loss_his[0:-1]):
                count = 0
                print(str(eph) + ' Val loss improve form!! :' + str(min(val_loss_his[0:-1])) + ' to ' + str(np.mean(loss_epoch_val)) )
                MODEL.append(model)
                # print('------------------Save Model!----------------')

            if np.mean(loss_epoch_val) > min(val_loss_his[0:-1]):
                # print('Val loss is not improve form ' + str(min(val_loss_his))  )
                count += 1
                if count == 20:
                    print('Stop Training')

                    return MODEL[-1]

                    break

In [814]:
epochs = 200
model = MLP_Regressor(X_train.shape[-1])

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.6,
                                     patience=5, min_lr= 0.00001 ,verbose = True)

model_trained = MLP_RegressorLoop(model, criterion, optimizer, scheduler, epochs,
                                    X_train, y_train, X_val, y_val)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([40])) that is different to the input size (torch.Size([40, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RMSE 65.70432
RMSE 65.66952
RMSE 65.63941
2 Val loss improve form!! :65.66951751708984 to 65.63941192626953
RMSE 65.59686
3 Val loss improve form!! :65.63941192626953 to 65.59686279296875
RMSE 65.5477
4 Val loss improve form!! :65.59686279296875 to 65.54769897460938
RMSE 65.48724
5 Val loss improve form!! :65.54769897460938 to 65.48724365234375
RMSE 65.41797
6 Val loss improve form!! :65.48724365234375 to 65.41796875
RMSE 65.33927
7 Val loss improve form!! :65.41796875 to 65.33927154541016
RMSE 65.2504
8 Val loss improve form!! :65.33927154541016 to 65.25039672851562
RMSE 65.15057
9 Val loss improve form!! :65.25039672851562 to 65.15057373046875
RMSE 65.03874
10 Val loss improve form!! :65.15057373046875 to 65.03874206542969
RMSE 64.91404
11 Val loss improve form!! :65.03874206542969 to 64.9140396118164
RMSE 64.775375
12 Val loss improve form!! :64.9140396118164 to 64.77537536621094
RMSE 64.62187
13 Val loss improve form!! :64.77537536621094 to 64.62187194824219
RMSE 64.452545
14 Val l

In [815]:
from sklearn.metrics import r2_score

model_trained.eval()
with torch.no_grad():
  prd = model_trained(X_test)

r2_score(y_test, prd)

-0.04546700627295408

# Using Copula to generate more data

In [816]:
import copulas
from copulas.datasets import sample_trivariate_xyz
from copulas.visualization import scatter_3d
from copulas.multivariate import GaussianMultivariate
from copulas.multivariate import VineCopula

# Train our gaussian copula
# Set train test validation
dftrain = df_train[selected_feas].reset_index(drop=True).values.tolist()
dftest  = df_test[selected_feas].reset_index(drop=True).values.tolist()

y_train = df_train['RelapseFreeSurvival (outcome)'].reset_index(drop=True)
y_test  = df_test['RelapseFreeSurvival (outcome)'].reset_index(drop=True)

In [817]:
copula_Gauss = GaussianMultivariate() # Gaussian Copula
# center = VineCopula('center') # Centered Vine Copula, the result is not good

copula_Gauss.fit(df_train[['RelapseFreeSurvival (outcome)'] + selected_feas].reset_index(drop=True))

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_continuous_distns.py:719: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.10/dist-packages/copulas/univariate/truncated_gaussian.py:45: RuntimeWarning: invalid value encountered in double_scalars
  a = (self.min - loc) / scale
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


In [818]:
synthetic_data = copula_Gauss.sample(500) # Generate more N data points
synthetic_data

,RelapseFreeSurvival (outcome),HER2,original_shape_Sphericity,original_firstorder_10Percentile,original_firstorder_Kurtosis,original_firstorder_Minimum,original_firstorder_Uniformity,original_glcm_ClusterShade,original_glcm_Id,original_glcm_Idm,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_glszm_GrayLevelNonUniformityNormalized,original_glszm_LowGrayLevelZoneEmphasis,original_glszm_SmallAreaEmphasis,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZonePercentage,original_ngtdm_Coarseness
0,67.793671,0.120001,0.588563,-0.264534,2.046217,-1.252844,0.711299,-0.292122,0.934185,0.934171,...,129.158586,0.007612,0.005819,0.748937,0.901473,0.316678,0.293502,0.332255,0.002511,-0.000803
1,55.368492,-0.041358,0.802420,0.225532,2.605173,-1.146696,0.872074,-0.077253,0.969605,0.969613,...,93.204554,0.005477,0.003352,0.720743,0.921199,0.509895,0.496765,0.504096,0.003169,0.018077
2,50.732327,0.077819,0.553041,0.363105,2.456239,-0.688691,0.929421,-0.053238,0.984392,0.984400,...,147.850302,0.005781,0.004419,0.945556,0.986847,0.495917,0.509000,0.488589,0.003315,0.001700
3,59.758731,-0.002395,0.398272,0.423077,2.783953,-1.083733,0.931342,-0.139172,0.978200,0.978197,...,150.574599,0.003253,0.002041,0.937495,0.987495,0.419341,0.413582,0.419672,0.001258,0.002541
4,111.820525,1.144811,0.809075,0.130991,2.556642,-1.214765,0.857108,-0.385353,0.954403,0.954418,...,137.542565,0.006049,0.004576,0.779180,0.935407,0.237973,0.263917,0.245121,0.002980,0.003816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,24.373074,0.173067,0.557655,0.217776,1.953500,-1.202115,0.901044,-0.275448,0.947793,0.947785,...,140.991726,0.009596,0.008531,1.031488,0.999895,0.016417,0.153765,0.015497,0.007061,-0.001933
496,53.069464,0.303505,0.565164,0.238843,2.409952,-1.115916,0.892971,-0.099128,0.968292,0.968260,...,129.731544,0.008846,0.006812,0.894818,0.970826,0.492104,0.479394,0.496167,0.007977,0.002446
497,59.519397,0.083567,0.499692,-0.209335,1.793549,-1.165925,0.740608,-0.420264,0.938376,0.938392,...,121.234075,0.009857,0.007757,0.894574,0.954347,0.523025,0.562118,0.513903,0.007464,-0.002518
498,22.669279,0.864690,0.435026,0.323773,4.204070,-1.396442,0.882193,-0.138241,0.989929,0.989941,...,172.907220,0.003878,0.002480,1.047349,0.997408,0.610989,0.677924,0.602197,0.002525,-0.027906


In [819]:
# Preprocess Categorical Data
synthetic_data = synthetic_data[synthetic_data['HER2'].apply(lambda x: x >= 0 and x <= 1)]
synthetic_data = synthetic_data[synthetic_data['HER2'].apply(lambda x: (1 - x) < 0.2 or (x < 0.2) )]

synthetic_data['HER2'] = synthetic_data['HER2'].apply(lambda x: 1 if x >= 0.8 else 0)

In [820]:
synthetic_data.reset_index(drop=True)

,RelapseFreeSurvival (outcome),HER2,original_shape_Sphericity,original_firstorder_10Percentile,original_firstorder_Kurtosis,original_firstorder_Minimum,original_firstorder_Uniformity,original_glcm_ClusterShade,original_glcm_Id,original_glcm_Idm,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,original_glszm_GrayLevelNonUniformityNormalized,original_glszm_LowGrayLevelZoneEmphasis,original_glszm_SmallAreaEmphasis,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZonePercentage,original_ngtdm_Coarseness
0,67.793671,0,0.588563,-0.264534,2.046217,-1.252844,0.711299,-0.292122,0.934185,0.934171,...,129.158586,0.007612,0.005819,0.748937,0.901473,0.316678,0.293502,0.332255,0.002511,-0.000803
1,50.732327,0,0.553041,0.363105,2.456239,-0.688691,0.929421,-0.053238,0.984392,0.984400,...,147.850302,0.005781,0.004419,0.945556,0.986847,0.495917,0.509000,0.488589,0.003315,0.001700
2,23.171456,0,0.312271,0.168994,2.381951,-1.379646,0.834250,-0.082329,0.977477,0.977461,...,120.227621,0.007663,0.005782,0.964179,0.976040,0.477667,0.456249,0.486912,0.005257,-0.028996
3,70.750028,1,0.688403,-0.500553,2.228686,-1.360715,0.675633,-0.174472,0.918232,0.918239,...,142.434918,0.011297,0.010744,1.018200,0.997568,0.432655,0.418638,0.441160,0.008658,-1.115495
4,57.448609,1,0.852764,-0.183131,2.655385,-1.732131,0.782588,-0.410228,0.960801,0.960811,...,156.577324,0.005572,0.004170,0.985724,0.992453,0.412173,0.419670,0.404472,0.005138,-0.006970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,23.407647,1,0.812281,0.239394,3.148149,-1.639297,0.863959,-0.525095,0.959918,0.959938,...,154.048712,0.002653,0.001330,0.905837,0.977157,0.484230,0.481092,0.487339,0.000453,0.000394
206,24.373074,0,0.557655,0.217776,1.953500,-1.202115,0.901044,-0.275448,0.947793,0.947785,...,140.991726,0.009596,0.008531,1.031488,0.999895,0.016417,0.153765,0.015497,0.007061,-0.001933
207,59.519397,0,0.499692,-0.209335,1.793549,-1.165925,0.740608,-0.420264,0.938376,0.938392,...,121.234075,0.009857,0.007757,0.894574,0.954347,0.523025,0.562118,0.513903,0.007464,-0.002518
208,22.669279,1,0.435026,0.323773,4.204070,-1.396442,0.882193,-0.138241,0.989929,0.989941,...,172.907220,0.003878,0.002480,1.047349,0.997408,0.610989,0.677924,0.602197,0.002525,-0.027906


In [821]:
# append the synthetic data with the original train data
df_train_syn = pd.concat([df_train[['RelapseFreeSurvival (outcome)'] + selected_feas].reset_index(drop=True), synthetic_data], ignore_index=True)

In [827]:
df_train_syn.to_csv('/content/Syntetic_train.csv', index = False)

In [822]:
X_train = torch.tensor(df_train_syn[df_train_syn.columns[1:]].values[50:], dtype=torch.float32)
X_val   = torch.tensor(df_train_syn[df_train_syn.columns[1:]].values[0:50], dtype=torch.float32)
X_test  = torch.tensor(dftest, dtype=torch.float32)

y_train = torch.tensor(df_train_syn['RelapseFreeSurvival (outcome)'].values[50:], dtype=torch.float32)
y_val   = torch.tensor(df_train_syn['RelapseFreeSurvival (outcome)'].values[0:50], dtype=torch.float32)
y_test  = torch.tensor(y_test.tolist(), dtype=torch.float32)

In [823]:
X_train.shape

torch.Size([472, 25])

In [824]:
from sklearn.metrics import r2_score

epochs = 200
model = MLP_Regressor(X_train.shape[-1])

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.6,
                                     patience=5, min_lr= 0.00001 ,verbose = True)

model_trained = MLP_RegressorLoop(model, criterion, optimizer, scheduler, epochs,
                                    X_train, y_train, X_val, y_val)

model_trained.eval()
with torch.no_grad():
  prd = model_trained(X_test)

r2_score(y_test, prd)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RMSE 69.340004
RMSE 69.294235
RMSE 69.2545
2 Val loss improve form!! :69.29423522949219 to 69.25450134277344
RMSE 69.210464
3 Val loss improve form!! :69.25450134277344 to 69.21046447753906
RMSE 69.16742
4 Val loss improve form!! :69.21046447753906 to 69.16741943359375
RMSE 69.1202
5 Val loss improve form!! :69.16741943359375 to 69.12020111083984
RMSE 69.06373
6 Val loss improve form!! :69.12020111083984 to 69.06372833251953
RMSE 68.99966
7 Val loss improve form!! :69.06372833251953 to 68.9996566772461
RMSE 68.92739
8 Val loss improve form!! :68.9996566772461 to 68.9273910522461
RMSE 68.84665
9 Val loss improve form!! :68.9273910522461 to 68.84664916992188
RMSE 68.756485
10 Val loss improve form!! :68.84664916992188 to 68.75648498535156
RMSE 68.655914
11 Val loss improve form!! :68.75648498535156 to 68.65591430664062
RMSE 68.54302
12 Val loss improve form!! :68.65591430664062 to 68.54302215576172
RMSE 68.41439
13 Val loss improve form!! :68.54302215576172 to 68.41439056396484
RMSE 68.2

-0.019606779170638466